In [1]:
%pip install pandas numpy scikit-learn tensorflow tqdm imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
version = 0

In [3]:
# Parameters
version = 0


# Models

The current models being implemented are namely:

* Supervised
    1. Logistic Regression Model
    2. Random Forest Classifier
    3. Support Vector Machine
    4. Deep Neural Network
    5. (New) Bayesian Network
    6. (New) Gradient Boosting

* Unsupervised
    1. (New) K-Means Clustering
    2. (New) Kernel Density Estimation

In [4]:
import importlib
import pandas as pd
import plotter
import numpy as np

# Selecting column names for CESD-10 Scale related features
cesd_col_names = ["_a_emobth", "_a_emomnd", "_a_emodep", "_a_emoeff", "_a_emohope",
                "_a_emofear", "_a_emoslp", "_a_emohap", "_a_emolone", "_a_emogo"]

cesd_col_names_w1 = ["w1" + col for col in cesd_col_names]
cesd_col_names_w2 = ["w2" + col for col in cesd_col_names]
cesd_col_names_w3 = ["w3" + col for col in cesd_col_names]
cesd_col_names_w4 = ["w4" + col for col in cesd_col_names]
cesd_col_names_w5 = ["w5" + col for col in cesd_col_names]

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

def print_correlations(df):
    correlation_df = df.drop(columns=['pid'])

    # Compute correlation matrix
    correlation_matrix = correlation_df.corr()

    # Display top of correlation matrix for quick review
    # print(correlation_matrix)

    num_features = 4
    check = 'score'

    # Get correlations with the 'depressed' feature
    depressed_corr = correlation_matrix[check].drop(check).drop('depressed')  # Drop self-correlation

    # Sort the correlations in descending order (absolute values can be used for both + and - correlations)
    top_10_corr = depressed_corr.abs().sort_values(ascending=False).head(num_features)
    cols = [i for i in top_10_corr.index.to_list()]
    cols.append(check)

    # Display the top 10 correlated features with 'depressed'
    print(f"Top {num_features} features most correlated with '{check}':")
    print(top_10_corr)

    # For specific features (e.g., top 5 features)
    sns.pairplot(correlation_df[cols])
    plt.show()


### Prepping dataset

In [6]:
df1 = pd.read_csv("CSV/wave1_select_labelled.csv")
df1 = df1.drop(columns=cesd_col_names_w1)
print(df1['pid'].count())

df2 = pd.read_csv("CSV/wave2_select_labelled.csv")
df2 = df2.drop(columns=cesd_col_names_w2)
print(df2['pid'].count())

df3 = pd.read_csv("CSV/wave3_select_labelled.csv")
df3 = df3.drop(columns=cesd_col_names_w3)
print(df3['pid'].count())

df4 = pd.read_csv("CSV/wave4_select_labelled.csv")
df4 = df4.drop(columns=cesd_col_names_w4)
print(df4['pid'].count())

df5 = pd.read_csv("CSV/wave5_select_labelled.csv")
df5 = df5.drop(columns=cesd_col_names_w5)
print(df5['pid'].count())

combined_df = pd.concat([df1, df2, df3, df4, df5], axis=0, ignore_index=True)
print(combined_df['pid'].count())
print(combined_df.columns, f"\n{len(combined_df.columns) - 1} columns")

# print(combined_df.describe())

14156


14589


14746


19294


18779
81564
Index(['pid', 'age', 'gender', 'race', 'marital_status', 'born_province',
       'employed', 'employed_take_home', 'employed_weekly_hours',
       'self_employed', 'self_employed_take_home',
       'self_employed_weekly_hours', 'casual_work', 'casual_weekly_hours',
       'highest_grade_school', 'tertiary_education', 'currently_enrolled',
       'fever', 'persistent_cough', 'cough_with_blood', 'chest_pain',
       'body_ache', 'headache', 'back_ache', 'joint_pain_arthritis',
       'diarrhoea', 'painful_urination', 'swelling_ankles',
       'severe_weight_loss', 'time_since_prev_consulation', 'had_tubercolosis',
       'had_high_blood_pressure', 'had_diabetes_or_high_blood_sugar',
       'had_stroke', 'had_asthma', 'had_heart_problems', 'had_cancer',
       'exercise_frequency', 'smokes_cigarettes', 'height_measurement',
       'weight_measurement', 'waist_measurement', 'score', 'depressed'],
      dtype='object') 
43 columns


In [7]:
correlation_df = combined_df.drop(columns=['pid'])

# Compute correlation matrix
correlation_matrix = correlation_df.corr()

# Display top of correlation matrix for quick review
# print(correlation_matrix)

num_features = 4
check = 'score'

# Get correlations with the 'depressed' feature
depressed_corr = correlation_matrix[check].drop(check).drop('depressed')  # Drop self-correlation

# Sort the correlations in descending order (absolute values can be used for both + and - correlations)
top_10_corr = depressed_corr.abs().sort_values(ascending=False).head(num_features)
cols = [i for i in top_10_corr.index.to_list()]
cols.append(check)

# Display the top 10 correlated features with 'depressed'
print(f"Top {num_features} features most correlated with '{check}':")
print(top_10_corr)
print(list(top_10_corr.index.to_list()))

# Set a threshold for strong correlations
threshold = 0.7

# Filter the correlation matrix
strong_corr = correlation_matrix[(correlation_matrix.abs() >= threshold) & (correlation_matrix != 1.0)]

print(f"Strong correlations (above {threshold} or below -{threshold}):")
# print(strong_corr)

# Extract upper triangle correlations
def get_highly_correlated_pairs(corr_matrix, threshold=0.8):
    corr_pairs = corr_matrix.unstack()
    high_corr_pairs = corr_pairs[(corr_pairs.abs() > threshold) & (corr_pairs != 1)].sort_values(ascending=False)
    return high_corr_pairs

# Get highly correlated pairs with threshold 0.8
high_corr_pairs = get_highly_correlated_pairs(correlation_matrix, threshold=0.7)

print("Highly correlated feature pairs:")
print(high_corr_pairs)

from sklearn.decomposition import PCA

# Standardize the data (mean=0, variance=1) before applying PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(correlation_df)

# Apply PCA
pca = PCA(n_components=5)  # You can set the number of principal components
pca_result = pca.fit_transform(scaled_data)

# Explained variance ratio of each component
print("Explained variance by each principal component:")
print(pca.explained_variance_ratio_)

import seaborn as sns
import matplotlib.pyplot as plt

# For specific features (e.g., top 5 features)
sns.pairplot(correlation_df[cols])
plt.show()



Top 4 features most correlated with 'score':
age                      0.133990
employed_weekly_hours    0.077481
marital_status           0.073908
height_measurement       0.066803
Name: score, dtype: float64
['age', 'employed_weekly_hours', 'marital_status', 'height_measurement']
Strong correlations (above 0.7 or below -0.7):
Highly correlated feature pairs:
swelling_ankles     painful_urination     0.821589
painful_urination   swelling_ankles       0.821589
cough_with_blood    had_cancer            0.816307
had_cancer          cough_with_blood      0.816307
                    had_stroke            0.810218
had_stroke          had_cancer            0.810218
had_cancer          had_heart_problems    0.791126
had_heart_problems  had_cancer            0.791126
score               depressed             0.790721
depressed           score                 0.790721
cough_with_blood    diarrhoea             0.771701
diarrhoea           cough_with_blood      0.771701
cough_with_blood    had_st

Explained variance by each principal component:
[0.29228669 0.06081021 0.0581849  0.04389335 0.03784196]


### Additional Preprocessing

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import plotter

combined_df = combined_df.drop(columns=["score"])

if version == 0:
    # Split data into features (X) and target (y)
    X = combined_df.drop('depressed', axis=1)
    y = combined_df['depressed']

    # Apply SMOTE for oversampling or RandomUnderSampler for undersampling
    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Or for undersampling
    # undersample = RandomUnderSampler()
    # X_resampled, y_resampled = undersample.fit_resample(X, y)

    # Combine back into a balanced dataframe
    df_balanced = pd.concat([X_resampled, y_resampled], axis=1)
    combined_df = df_balanced

    print(combined_df['pid'].count())

    plotter.plot_bar(combined_df['depressed'], "Distribution of depression count after sampling")

if version == 1:
    # Separate majority and minority classes
    majority_class = combined_df[combined_df['depressed'] == 0]
    minority_class = combined_df[combined_df['depressed'] == 1]

    # Oversample the minority class to match the number of majority class samples
    oversampled_minority = minority_class.sample(len(majority_class), replace=True)

    # Concatenate the majority class with the oversampled minority class
    df_balanced = pd.concat([majority_class, oversampled_minority])

    # Shuffle the dataset
    df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

    combined_df = df_balanced

    plotter.plot_bar(combined_df['depressed'], "Distribution of depression count after sampling")

121936


/home-mscluster/tdremendos/models/plotter.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(value_counts.index, rotation=rotation)


Precision = TP / TP + FP (How often are positive predictions correct?)
Recall = TP / TP + FN (Can an ML model find all instances of the positive class?)

In [9]:
OPTIMIZE = True

### Logistic Regression

In [10]:
import logisticRegression
importlib.reload(logisticRegression)

LR = logisticRegression.LogisticRegressionModel(combined_df, combined_df['depressed'])

LR.run(0.5)

# Parameter grid for GridSearchCV
param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['l1', 'l2']
}

if OPTIMIZE:
    LR.optimize_params(param_grid, "models/LR_optimized_model.pkl")
    LR.run(0.5)

TRAINING MODEL: 


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           42     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  1.04160D+00


 This problem is unconstrained.



At iterate   50    f=  6.39167D-01    |proj g|=  3.14239D-01



At iterate  100    f=  6.32858D-01    |proj g|=  8.30588D-02


...


At iterate  800    f=  6.28008D-01    |proj g|=  5.82751D-04



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   42    849    964      1     0     0   9.715D-05   6.280D-01
  F =  0.62800780084412633     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
SAVING EVALUATION TO:  metrics/LR_metrics.pkl
Validation Confusion Matrix:
[[8111 4224]
 [4185 7867]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.66      0.66     12335
           1       0.65      0.65      0.65     12052

    accuracy                           0.66     24387
   macro avg       0.66      0.66      0.66     24387
weighted avg   

SAVING EVALUATION TO:  metrics/LR_metrics.pkl
Test Confusion Matrix:
[[7944 4303]
 [4178 7963]]
Test Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.65      0.65     12247
           1       0.65      0.66      0.65     12141

    accuracy                           0.65     24388
   macro avg       0.65      0.65      0.65     24388
weighted avg       0.65      0.65      0.65     24388

Test Accuracy: 0.65
Test F1 Score: 0.65
Test Precision: 0.65
Test Recall: 0.66
Test AUC-ROC: 0.70

SAVING MODEL TO:  models/LR_model.pkl
Fitting 5 folds for each of 12 candidates, totalling 60 fits


Best Parameters: {'logisticregression__C': 1, 'logisticregression__penalty': 'l1'}
Best Score: 0.6560462139153332

Model Parameters:  {'cv': 5, 'error_score': nan, 'estimator__memory': None, 'estimator__steps': [('standardscaler', StandardScaler()), ('logisticregression', LogisticRegression(max_iter=1000, solver='saga', tol=0.1))], 'estimator__verbose': False, 'estimator__standardscaler': StandardScaler(), 'estimator__logisticregression': LogisticRegression(max_iter=1000, solver='saga', tol=0.1), 'estimator__standardscaler__copy': True, 'estimator__standardscaler__with_mean': True, 'estimator__standardscaler__with_std': True, 'estimator__logisticregression__C': 1.0, 'estimator__logisticregression__class_weight': None, 'estimator__logisticregression__dual': False, 'estimator__logisticregression__fit_intercept': True, 'estimator__logisticregression__intercept_scaling': 1, 'estimator__logisticregression__l1_ratio': None, 'estimator__logisticregression__max_iter': 1000, 'estimator__logisti

SAVING EVALUATION TO:  metrics/LR_optimized_metrics.pkl
Validation Confusion Matrix:
[[8159 4176]
 [4188 7864]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.66      0.66     12335
           1       0.65      0.65      0.65     12052

    accuracy                           0.66     24387
   macro avg       0.66      0.66      0.66     24387
weighted avg       0.66      0.66      0.66     24387

Validation Accuracy: 0.66
Validation F1 Score: 0.65
Validation Precision: 0.65
Validation Recall: 0.65
Validation AUC-ROC: 0.71

SAVING EVALUATION TO:  metrics/LR_optimized_metrics.pkl
Test Confusion Matrix:
[[7980 4267]
 [4189 7952]]
Test Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.65      0.65     12247
           1       0.65      0.65      0.65     12141

    accuracy                           0.65     24388
   macro avg       0.65      0.65      0.65   

### Random Forest

In [11]:
import randomForest
from sklearn.model_selection import ParameterGrid

importlib.reload(randomForest)

RF = randomForest.RandomForestModel(combined_df, combined_df['depressed'])

param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy', 'log_loss'] 
        }

RF.run()

if OPTIMIZE:
    RF.optimize_params(param_grid=param_grid, path="models/RF_optimized_model.pkl")
    RF.run()

TRAINING MODEL: 


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.9s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s


SAVING EVALUATION TO:  metrics/RF_metrics.pkl
Validation Confusion Matrix:
[[11948   387]
 [ 3791  8261]]
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.97      0.85     12335
           1       0.96      0.69      0.80     12052

    accuracy                           0.83     24387
   macro avg       0.86      0.83      0.82     24387
weighted avg       0.86      0.83      0.82     24387

Validation Accuracy: 0.83
Validation F1 Score: 0.80
Validation Precision: 0.96
Validation Recall: 0.69
Validation AUC-ROC: 0.87



...

Testing: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}:   8%|▊         | 27/324 [25:57<5:11:28, 62.93s/it] 

Testing: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}:   9%|▊         | 28/324 [25:57<4:42:28, 57.26s/it]

Testing: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}:   9%|▊         | 28/324 [27:22<4:42:28, 57.26s/it]

Testing: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}:   9%|▉         | 29/324 [27:22<5:22:56, 65.68s/it]

Testing: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}:   9%|▉         | 29/324 [29:30<5:22:56, 65.68s/it]

### Support Vector Machine

In [ ]:
import supportVectorMachine
importlib.reload(supportVectorMachine)

SVM = supportVectorMachine.SVMModel(combined_df, combined_df['depressed'])

SVM.run()

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'degree': [2, 3, 4],  # Only used by poly kernel
    'decision_function_shape': ['ovo', 'ovr'] 
}

if OPTIMIZE:
    SVM.optimize_params(param_grid, "models/SVM_optimized_model.pkl")
    SVM.run()

### Deep Neural Network

In [ ]:
import neuralNetwork
importlib.reload(neuralNetwork)

NN = neuralNetwork.DeepNeuralNetworkModel(combined_df, combined_df['depressed'])

NN.run(0.5)

param_grid = {
    'neurons': [32, 64, 128],
    'layers': [3, 4],
    'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01],
    'batch_size': [32, 64],
    'epochs': [500]
}

if OPTIMIZE:
    NN.optimize_params(param_grid, "models/NN_optimized_model.pkl")
    NN.run(0.5)

# Comparing Evaluations

In [ ]:
importlib.reload(plotter)
importlib.reload(np)

path = "metrics/LR_metrics.pkl"

metrics = plotter.load_model(path)
LR_conf_matrix, LR_class_report, LR_accuracy, LR_f1, LR_precision, LR_recall, LR_auc_roc = metrics

path = "metrics/NN_metrics.pkl"

metrics = plotter.load_model(path)
NN_conf_matrix, NN_class_report, NN_accuracy, NN_f1, NN_precision, NN_recall, NN_auc_roc = metrics

path = "metrics/RF_metrics.pkl"

metrics = plotter.load_model(path)
RF_conf_matrix, RF_class_report, RF_accuracy, RF_f1, RF_precision, RF_recall, RF_auc_roc = metrics

path = "metrics/SVM_metrics.pkl"

metrics = plotter.load_model(path)
SVM_conf_matrix, SVM_class_report, SVM_accuracy, SVM_f1, SVM_precision, SVM_recall, SVM_auc_roc = metrics

metrics = ['Accuracy', 'F1', 'Precision', 'Recall', 'AUC_ROC']
models = ['LR', 'NN', 'RF', 'SVM']
scores = pd.DataFrame({"LR": [LR_accuracy, LR_f1, LR_precision, LR_recall, LR_auc_roc], 
                           "NN": [NN_accuracy, NN_f1, NN_precision, NN_recall, NN_auc_roc], 
                           "RF": [RF_accuracy, RF_f1, RF_precision, RF_recall, RF_auc_roc], 
                           "SVM": [SVM_accuracy, SVM_f1, SVM_precision, SVM_recall, SVM_auc_roc]},
                           index=metrics)

print(scores)

# Plotting the comparison
plotter.plot_model_comparison(scores, title="Model Performance", rotation=0, group_by_model=True)

confusions = pd.DataFrame({'LR': np.array(LR_conf_matrix).flatten(),
                           'NN': np.array(NN_conf_matrix).flatten(),
                           'RF': np.array(RF_conf_matrix).flatten(),
                           'SVM': np.array(SVM_conf_matrix).flatten()})

# Plot the confusion matrix heatmaps
plotter.plot_confusion_matrix_heatmaps(confusions, title='Model Confusion Matrix Comparisons', solo=True)